# SLB Test Tool

## Setup

### Install Selenium

In [4]:
from IPython.display import clear_output

%pip install selenium 
clear_output(wait=True)

%pip install webdriver_manager 
clear_output(wait=True)

%pip install beautifulsoup4
clear_output(wait=True)

%pip install pandas 
clear_output(wait=True)

%pip install ipywidgets
clear_output()

print("All packages installed")

All packages installed


In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
from collections import defaultdict
from xml.sax.saxutils import quoteattr
from IPython.display import display
from ipywidgets import HTML

import ipywidgets as widgets
import pandas as pd
import time
import json

prefs = {"profile.default_content_setting_values.notifications": 1}
chrome_options = Options()
#chrome_options.add_argument("--headless")
chrome_options.add_experimental_option("prefs", prefs)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.quit()

def new_window(url, wait=0):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)
    time.sleep(wait)
    return driver

def highlight_element(driver, selector):
    script = f"""
    var element = document.evaluate('{selector}', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
    if (element) {{
        var originalStyle = element.style.border;
        var count = 0;
        var interval = setInterval(function() {{
            count += 1;
            element.style.border = count % 2 ? '3px solid red' : '';
            if (count > 9) {{
                clearInterval(interval);
                element.style.border = originalStyle;
            }}
        }}, 500);
    }}
    """
    try:
        driver.execute_script(script)
    except Exception as e:
        print("Error while highlighting the element:", e)


clear_output()
print("Done!")

Done!


## Source Code

### Generate Objects Keys

In [6]:
def generate_key(tag, element, counter):
    base = {"input": "txt", "button": "btn", "a": "lnk", "img": "img"}
    prefix = base.get(tag, "elem")

    parts = [prefix]
    
    txt = element.get_text(strip=True)
    if txt:
        parts.append(txt.replace(" ", "_"))
        return key_builder(parts, counter)
    
    if element.get('name'):
        parts.append(element['name'])
        return key_builder(parts, counter)
    
    if element.get('aria-label'):
        parts.append(element['aria-label'].replace(" ", "_"))
        return key_builder(parts, counter)

    if element.get('id'):
        parts.append(element['id'])
        return key_builder(parts, counter)
    
    if element.get('data-test-id'):
        parts.append(element['data-test-id'])
        return key_builder(parts, counter)
    
    return key_builder(parts, counter)


def key_builder(parts, counter):
    key = '_'.join(filter(None, parts)).lower().replace("-", "_")
    counter[key] += 1
    return key if counter[key] == 1 else f"{key}_{counter[key]}"

print("Done!")

Done!


### Generate Objects Selectors

In [7]:
def generate_selector(element, driver):
    selectors = []

    for attr in ['id', 'name', 'placeholder', 'aria-label', 'data-test-id', 'alt']:
        if element.get(attr):
            value = quoteattr(element[attr])
            selectors.append(f"@{attr}={value}")


    if selectors:
        xpath_selector = f"//{element.name}[{' and '.join(selectors)}]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector


    txt = element.get_text(strip=True)
    if txt:
        xpath_selector = f"//{element.name}[contains(text(), {quoteattr(txt)})]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector


    if element.get('class'):
        classes = '.'.join(element.get('class'))
        xpath_selector = f"//{element.name}[contains(@class, {quoteattr(classes)})]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector

    return None

print("Done!")

Done!


### Generate Objects Selectors

In [8]:

def read_page_objects_metadata(driver):

    html = driver.page_source.encode("utf-8")
    soup = BeautifulSoup(html, 'html.parser')
    elements = soup.find_all(['input', 'button', 'a', 'img'])

    counter = defaultdict(int)
    result = defaultdict(list)

    for element in elements:
        tag = element.name
        selector = generate_selector(element, driver)
        key = generate_key(tag, element, counter)

        result[tag].append({
            "key": key,
            "selector": selector,
            "attributes": {attr: element.get(attr) for attr in element.attrs},
            "text": element.get_text(strip=True),
            "visibility": "visible" if element.get('type') != 'hidden' else "invisible"
        })
    return result

## GUI

### Toolbar

In [9]:
global toolbar
def new_toolbar():
    
    global new_window_button, metadata_button, filter_text, export_button

    new_window_button = widgets.Button(description="New Window") 
    metadata_button = widgets.Button(description="Inspect") 
    filter_text = widgets.Text(placeholder='Type to filter...') 
    export_button = widgets.Button(description="Export Selected")
    display(widgets.HBox([new_window_button, metadata_button, filter_text, export_button]))

toolbar = new_toolbar()
display(toolbar)


None

### Table Header

In [33]:
global table
def new_table(df=None):

    columns = [
        ("Export", 30), 
        ("Web", 30), 
        ("Data", 30),
        ("Alias", 150), 
        ("Text", 200), 
        ("Type", 0), 
        ("Att", 0)
    ]

    if df is None: 
        df = pd.DataFrame(columns=[col_name for col_name, _ in columns])

    lines = len(df)+1
    grid = widgets.GridspecLayout(lines+1, len(columns)+1)

    for i in range(lines):
        for j, (col_name, col_width) in enumerate(columns):

            if col_width <= 0: continue

            if i == 0:
                grid[i, j] = widgets.HTML(value=f"<b>{col_name}</b>", layout=widgets.Layout(width=f'{col_width}px'))
                continue

            if col_name == "Text":
                grid[i, j] = widgets.Label(value=str(cell_value), layout=widgets.Layout(width='auto'))    
                continue

            if col_name == "Export":
                grid[i, j] = widgets.Button(description="N", layout=widgets.Layout(width='50px', height='30px'))
                continue

            if col_name == "Data":
                grid[i, j] = widgets.Button(icon='table', layout=widgets.Layout(width='50px', height='30px'))
                continue

            cell_value = getattr(df.iloc[i-1], col_name, '')
            if col_name == "Web":
                grid[i, j] = widgets.Button(tooltip=f"{cell_value}", icon='eye', layout=widgets.Layout(width='50px', height='30px'))
                continue

            grid[i, j] = widgets.Label(value=str(cell_value), layout=widgets.Layout(width='auto'))

    return grid


sample = pd.DataFrame({
    "Export": [True, False],
    "Alias": ["btn_cancel", "btn_ok"],
    "Web": ["Selector01", "Selector02"],
    "Text": ["Cancel", "Ok"],
    "Type": ["button", "button"],
    "Att": ["", ""],
    "Data": ["", ""]
})


table = new_table(sample)
display(table)


GridspecLayout(children=(HTML(value='<b>Export</b>', layout=Layout(grid_area='widget001', width='30px')), HTML…

In [11]:
global df, test_window
test_window = None

def start_playground():

    global test_window, df
    df = pd.DataFrame(columns=["Export", "Alias", "Web", "Type", "Text", "Att", "Data"])
    df_original = pd.DataFrame()


    def on_new_window_clicked(b=None):
        global test_window
        try: 
            test_window.quit()
        except: 
            pass  # ignore exceptions
        test_window = new_window("https://life.stg.wellzesta.com/login")


    def on_metadata_clicked(b):
        clear_output(wait=True)
        global test_window, df
        if test_window is None:
            on_new_window_clicked()

        metadata = read_page_objects_metadata(test_window)
        rows = []
        for tag_name, elements in metadata.items():
            for element in elements:

                att = element.pop('attributes', {})
                clazz =  att.pop('class', [])

                for k, v in att.items():
                    element[k] = v

                element['class'] = clazz
                rows.append({
                    "Export": False,
                    "Alias": element.get('key', ''),
                    "Web": element.get('selector', ''),
                    "Type": tag_name,
                    "Text": element.get('text', ''),
                    "Data": element,
                })

        df = pd.DataFrame(rows)
        update_table()
        print(f"Loaded from {test_window.current_url}")


    def on_export_button_clicked(b):
        print("Exporting...")



    def adjust_column_widths(df, base_width=8, padding=10):
        widths = {}
        for col in df.columns:
            if col == "Export" or col == "Web": 
                widths[col] = 50
                continue
            
            max_len = df[col].astype(str).map(len).max()  # Comprimento máximo do conteúdo na coluna
            widths[col] = min(max_len * base_width + padding, 400)  # Calcular a largura com um máximo para evitar colunas muito largas
        return widths
    

    def on_export_toggle(b, index):
        global df
        df.at[index, 'Export'] = not df.at[index, 'Export']
        update_table()


    def print_data(element):
        clear_output(wait=True) 
        update_table() 
        display(element)


    def create_properties_widget():
        properties_widget = widgets.Output()
        with properties_widget:
            clear_output(wait=True)
        return properties_widget


    global properties_widget
    properties_widget = create_properties_widget()


    def print_data(data):
        clear_output(wait=True)
        update_table()
        display(data)


    def update_table():
        global df, test_window, properties_widget
        nonlocal export_button

        if df.empty:
            clear_output(wait=True)
            display(widgets.HBox([new_window_button, metadata_button]))
            return

        df.sort_values(by=['Export', 'Type', 'Alias'], ascending=[False, True, True], inplace=True)
        #df = df.drop(columns=["Attributes", "Text"], errors='ignore')

        clear_output(wait=True)
        display(widgets.HBox([new_window_button, metadata_button, filter_text, export_button]))

        # Calcular a largura das outras colunas
        col_widths = adjust_column_widths(df)
        grid = widgets.GridspecLayout(len(df)+1, len(df.columns))

        # Cabeçalhos
        for i, col in enumerate(df.columns):
            if col == "Type": continue 
            grid[0, i] = HTML(value=f"<b>{col}</b>", layout=widgets.Layout(width=f'{col_widths[col]}px'))

        # Dados, botões de exportação e botões com seletores
        for i, (index, row) in enumerate(df.iterrows(), start=1):

            for j, col in enumerate(df.columns):
                if col == "Type": continue

                bt_layout=widgets.Layout(width='50px', height='30px')
                if col == "Export":
                    export_button_text = 'Y' if row[col] else 'N'
                    export_button = widgets.Button(
                        description=export_button_text, 
                        layout=bt_layout
                    )
                    export_button.on_click(lambda b, index=index: on_export_toggle(b, index))
                    grid[i, j] = export_button
                    continue

                if col == "Web":
                    button_icon = 'eye' 
                    selector_text = row[col] if row[col] else 'No Selector'
                    disabled = False if row[col] else True
                    highlight_button = widgets.Button(
                        disabled=disabled,
                        description='', 
                        tooltip=selector_text, 
                        icon=button_icon, 
                        layout=bt_layout
                    )
                    highlight_button.on_click(lambda b, selector=row['Web']: highlight_element(test_window, selector))
                    grid[i, j] = highlight_button
                    continue

                if col == "Data":
                    button_icon = 'table' 
                    selector_text = row[col]

                    data_button = widgets.Button(
                        icon=button_icon, 
                        layout=bt_layout
                    )

                    data_button.on_click(lambda b, row=row: print_data(row[col]))
                    grid[i, j] = data_button
                    continue

                grid[i, j] = widgets.Label(value=str(row[col]), layout=widgets.Layout(width=f'{col_widths[col]}px'))

        display(grid)


    # Widgets
    new_window_button = widgets.Button(description="New Window")
    metadata_button = widgets.Button(description="Inspect")
    filter_text = widgets.Text(placeholder='Type to filter...')
    export_button = widgets.Button(description="Export Selected")
    

    # Eventos
    new_window_button.on_click(on_new_window_clicked)
    metadata_button.on_click(on_metadata_clicked)
    export_button.on_click(on_export_button_clicked)


    # Inicializa a interface sem dados
    update_table()
    





## Playground

In [12]:
README = """
When a DEV loves a QA
Can't keep his mind on any code
He'd debug all the errors
For the quality she's known

Yes, she's tough, oh so demanding
But he's the champ, codes so well
Even writes a testing framework
Just to make her world excel

Turns his back on his shortcuts
If they don't pass her test
When a DEV loves a QA
He's precise in every line
Crafting codes she envisioned

He'd give up all his shortcuts
And embrace the cleaner code
If she said that's the way
It ought to be

Though she's strict, he's so adept
Together they're a code-quality duet
"""

start_playground()